In [10]:
import numpy as np
import cv2

In [11]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png').astype(np.float32)
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [12]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------
    alpha, beta = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    h, w = depth.shape
    u, v = np.meshgrid(np.arange(w), np.arange(h))
    u = u.astype(np.float32)
    v = v.astype(np.float32)
    z = depth
    x = (u - cx) * z / alpha
    y = (v - cy) * z / beta
    points = np.stack((x, y, z), axis=-1)
    pc = points[seg.astype(bool)]
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [26]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------
dists = np.linalg.norm(partial_pc_sampled[:, np.newaxis] - full_pc_sampled, axis=-1)
min_dists = np.min(dists, axis=1)
one_way_CD = np.mean(min_dists)
print('one way chamfer distance: ', one_way_CD)

# For submission
one_way_CD = np.array([one_way_CD])
np.savetxt('../results/one_way_CD.txt', one_way_CD)
# Answer: 0.009976257336995639

one way chamfer distance:  0.009630624100727467
